In [1]:
import os
import re
import string
import numpy as np

import tensorflow as tf
from tensorflow.keras import layers
from sklearn.metrics import f1_score

In [2]:
import pandas as pd
import tqdm

df = pd.read_csv('./test.csv')

df.head(5)

,Song,Song year,Artist,Genre,Lyrics,Track_id
0,craftsmanship,2005,buck-65,Hip-Hop,Most folks spend their days daydreaming of fin...,8294
1,come-on-out,2012,the-elwins,Indie,Take your cold hands and put them on my face\n...,21621
2,riot,2013,bullet-for-my-valentine,Metal,Are you ready it's time for war\nWe'll break d...,3301
3,that-s-what-girls-do,2007,dream-street,Pop,You ask me why I change the color of my hair\n...,2773
4,believe-in-a-dollar,2012,cassidy,Hip-Hop,Do you believe in magic in a young girl's hear...,16797


In [3]:
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
import re

# Preprocessing
def prepocessing(lyrics, remove_stopwords=False, stops=set(stopwords.words('english'))):
    lyric_text = BeautifulSoup(lyric, "html5lib").get_text()
    lyric_text = re.sub("[^a-zA-Z]", " ", lyric_text)
    lyric_text = lyric_text.lower()
    
    lyric_words = lyric_text.split()
    if remove_stopwords:
        lyric_words = lyric_text.split()
        lyric_words = [w for w in lyric_words if not w in stops]
        
    lyric_text = ' '.join(lyric_words)
    return lyric_text

In [4]:
processed_lyrics = []
for lyric in tqdm.tqdm(df['Lyrics'].values):
    processed = prepocessing(lyric)
    processed_lyrics.append(processed)

100%|████████████████████████████████████████████████████████████████████████████| 7935/7935 [00:03<00:00, 2403.82it/s]


In [5]:
class_code = {
    'Rock': 0,
    'Metal': 1,
    'Pop': 2,
    'Indie': 3,
    'Folk': 4,
    'Electronic': 5,
    'R&B': 6,
    'Jazz': 7,
    'Hip-Hop': 8,
    'Country': 9,
}

In [6]:
import numpy as np
from sklearn.model_selection import train_test_split

y = []
for genre in df['Genre']:
    y.append(class_code[genre])

x = processed_lyrics
y = np.array(y)

# Get F-1 Score

## Experiment 1
- embedding dim = 128
- one bidirection layer
- weak class weight (softmax)

In [8]:
model = tf.keras.models.load_model('./experiment1-LSTM64', custom_objects={'LSTMCell': tf.keras.layers.LSTM})

In [10]:
from sklearn.metrics import classification_report

y_pred = model.predict(x, batch_size=32, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)

print(classification_report(y, y_pred_bool))

248/248 [==============================] - 240s 963ms/step
              precision    recall  f1-score   support

           0       0.26      0.30      0.27      1410
           1       0.04      0.25      0.07       810
           2       0.06      0.04      0.05      1110
           3       0.05      0.02      0.03       510
           4       0.19      0.08      0.12       495
           5       0.00      0.00      0.00       660
           6       0.00      0.00      0.00       510
           7       0.06      0.00      0.01       660
           8       0.00      0.00      0.00       960
           9       0.00      0.00      0.00       810

    accuracy                           0.09      7935
   macro avg       0.07      0.07      0.05      7935
weighted avg       0.08      0.09      0.07      7935



C:\Users\ltw97\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ltw97\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ltw97\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Experiment 2
- embedding dim = 300
- one Bidirectional
- weak class weight

In [11]:
model = tf.keras.models.load_model('./experiment2-LSTM64', custom_objects={'LSTMCell': tf.keras.layers.LSTM})

In [12]:
from sklearn.metrics import classification_report

y_pred = model.predict(x, batch_size=32, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)

print(classification_report(y, y_pred_bool))

248/248 [==============================] - 235s 944ms/step
              precision    recall  f1-score   support

           0       0.26      0.45      0.33      1410
           1       0.04      0.16      0.06       810
           2       0.05      0.04      0.04      1110
           3       0.04      0.02      0.03       510
           4       0.20      0.24      0.22       495
           5       0.00      0.00      0.00       660
           6       0.00      0.00      0.00       510
           7       0.03      0.00      0.01       660
           8       0.00      0.00      0.00       960
           9       0.00      0.00      0.00       810

    accuracy                           0.12      7935
   macro avg       0.06      0.09      0.07      7935
weighted avg       0.07      0.12      0.09      7935



C:\Users\ltw97\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ltw97\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ltw97\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Experiment 2-2

In [13]:
model = tf.keras.models.load_model('./experiment2-2bidriection', custom_objects={'LSTMCell': tf.keras.layers.LSTM})

In [14]:
from sklearn.metrics import classification_report

y_pred = model.predict(x, batch_size=32, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)

print(classification_report(y, y_pred_bool))

248/248 [==============================] - 505s 2s/step
              precision    recall  f1-score   support

           0       0.24      0.38      0.30      1410
           1       0.03      0.20      0.06       810
           2       0.04      0.03      0.03      1110
           3       0.04      0.02      0.02       510
           4       0.16      0.06      0.09       495
           5       0.00      0.00      0.00       660
           6       0.00      0.00      0.00       510
           7       0.00      0.00      0.00       660
           8       0.00      0.00      0.00       960
           9       0.00      0.00      0.00       810

    accuracy                           0.10      7935
   macro avg       0.05      0.07      0.05      7935
weighted avg       0.06      0.10      0.07      7935



C:\Users\ltw97\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ltw97\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ltw97\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Experiment 2-2-1

In [15]:
model = tf.keras.models.load_model('./exp2-2-1', custom_objects={'LSTMCell': tf.keras.layers.LSTM})

In [16]:
from sklearn.metrics import classification_report

y_pred = model.predict(x, batch_size=32, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)

print(classification_report(y, y_pred_bool))

248/248 [==============================] - 244s 980ms/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1410
           1       0.11      0.54      0.19       810
           2       0.00      0.00      0.00      1110
           3       0.05      0.04      0.04       510
           4       0.13      0.48      0.20       495
           5       0.10      0.16      0.12       660
           6       0.00      0.00      0.00       510
           7       0.11      0.00      0.00       660
           8       0.03      0.02      0.03       960
           9       0.80      0.00      0.01       810

    accuracy                           0.10      7935
   macro avg       0.13      0.12      0.06      7935
weighted avg       0.13      0.10      0.05      7935



C:\Users\ltw97\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ltw97\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ltw97\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Experiment 2-2-2

In [17]:
model = tf.keras.models.load_model('./exp2-2-2', custom_objects={'LSTMCell': tf.keras.layers.LSTM})

In [18]:
from sklearn.metrics import classification_report

y_pred = model.predict(x, batch_size=32, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)

print(classification_report(y, y_pred_bool))

248/248 [==============================] - 506s 2s/step
              precision    recall  f1-score   support

           0       0.23      0.01      0.02      1410
           1       0.07      0.46      0.12       810
           2       0.06      0.04      0.05      1110
           3       0.06      0.03      0.04       510
           4       0.13      0.28      0.17       495
           5       0.11      0.05      0.07       660
           6       0.00      0.00      0.00       510
           7       0.17      0.00      0.00       660
           8       0.04      0.01      0.01       960
           9       0.35      0.02      0.04       810

    accuracy                           0.08      7935
   macro avg       0.12      0.09      0.05      7935
weighted avg       0.13      0.08      0.05      7935



## Experiment 2-2-3

In [19]:
model = tf.keras.models.load_model('./exp2-2-3', custom_objects={'LSTMCell': tf.keras.layers.LSTM})

In [20]:
from sklearn.metrics import classification_report

y_pred = model.predict(x, batch_size=32, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)

print(classification_report(y, y_pred_bool))

248/248 [==============================] - 494s 2s/step
              precision    recall  f1-score   support

           0       0.23      0.50      0.31      1410
           1       0.02      0.09      0.04       810
           2       0.06      0.05      0.05      1110
           3       0.05      0.03      0.04       510
           4       0.29      0.08      0.13       495
           5       0.00      0.00      0.00       660
           6       0.00      0.00      0.00       510
           7       0.05      0.01      0.02       660
           8       0.00      0.00      0.00       960
           9       0.00      0.00      0.00       810

    accuracy                           0.11      7935
   macro avg       0.07      0.08      0.06      7935
weighted avg       0.08      0.11      0.08      7935



C:\Users\ltw97\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ltw97\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ltw97\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
